# Drive Test Tag Generation
Generate tags for the written portion of the chinese driving exam using a process similar to BERTopic.

## 1. Load Data
Loading data from a local database into a question bank class.

In [1]:
from src.qb.question_bank import QuestionBank
from data_storage.database.json_database import LocalJsonDB

In [2]:
db = LocalJsonDB("data_storage/database/json_db/data.json",
                 "data_storage/database/json_db/images")
qb : QuestionBank = db.load()
print(qb.question_count())

2836


## 2. Format Data
Although the Siglip2 model can handle images of different sizes, we will still standardize the image sizes to the same size to avoid unnecessary complications.

In [3]:
from data_cleaning.img_reshaper import ImgSquarer

In [4]:
IMG_DIR_256 = "data_cleaning/resized_imgs/img256"
squarer_256 = ImgSquarer(256)

In [5]:
def resize_images(qb: QuestionBank, squarer: ImgSquarer, new_dir: str) -> None:
    for chapter_id in qb.list_chapters():
        for qid in qb.get_qids_by_chapter(chapter_id):
            question = qb.get_question(qid)
            if question.get_img_path() is not None:
                question.set_img_path(squarer.reshape(qid, qb.get_img_dir(), new_dir))

In [6]:
import os

In [7]:
# If the directory is empty, resize images.
if not os.listdir(IMG_DIR_256):
    print("Resizing images to 256x256...")
    resize_images(qb, squarer_256, IMG_DIR_256)
else:
    print("Images already resized to 256x256, skipping...")

Images already resized to 256x256, skipping...


## 3. Create Multimodal Embeddings
Create multimodal embeddings for the questions using a Siglip2 model.

In [8]:
# Library Imports
from transformers import AutoModel, AutoProcessor

# Local Imports
from embedder.siglip2_qb_embedder import Siglip2QBEmbedder

### a) Load/Download the Siglip2 Model
We will be using "google/siglip2-base-patch16-256" for this task.

In [9]:
import torch

In [10]:
MODEL_NAME = "google/siglip2-base-patch16-256"
model = AutoModel.from_pretrained(MODEL_NAME, torch_dtype=torch.float32, attn_implementation="sdpa")
processor = AutoProcessor.from_pretrained(MODEL_NAME, use_fast=True)

### b) Create embeddings

#### i) Define a logger

In [11]:
import logging
from logging import Logger
from datetime import datetime

LOGGING_PATH = "logs"

def get_logger(name: str) -> Logger:
    # Create logger
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO) # Set the logging level

    # Create a file handler with timestamp in filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    file_handler = logging.FileHandler(
        os.path.join(LOGGING_PATH, f"{name}_{timestamp}.log")
    )

    # Create formatter
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    file_handler.setFormatter(formatter)

    # Add handler to logger
    logger.addHandler(file_handler)

    return logger

embedder_logger = get_logger("embedder")

#### ii) Create the embedder

In [12]:
custom_embedder = Siglip2QBEmbedder(model, processor, embedder_logger)

#### iii) Generate embeddings

In [13]:
import numpy as np

In [14]:
def save_embeddings(embeddings, file_path):
    np.savez(file_path, **{str(qid): embeddings[qid] for qid in embeddings})

In [15]:
EMBEDDING_FILE_NAME = "data_storage/embedding_dir/siglip2_embeddings.npz"
if os.path.exists(EMBEDDING_FILE_NAME):
    print(f"Embeddings already exist at {EMBEDDING_FILE_NAME}, skipping generation.")
else:
    print("Generating embeddings...")
    embeddings = custom_embedder.encode_qb(qb)

    print(f"Saving embeddings to {EMBEDDING_FILE_NAME}...")
    save_embeddings(embeddings, EMBEDDING_FILE_NAME)

Embeddings already exist at data_storage/embedding_dir/siglip2_embeddings.npz, skipping generation.


## 4. Dimension Reduction

### a) Load Embeddings

In [16]:
def load_embeddings(file_path):
    loaded = np.load(file_path)
    return {key: loaded[key] for key in loaded.files}

In [17]:
id_to_embedding = load_embeddings(EMBEDDING_FILE_NAME)

In [18]:
from typing import List

from embedder.siglip2_qb_embedder import format_question

In [19]:
def format_for_clustering(id_to_embedding: dict, qb: QuestionBank) -> (List[str], List[str], np.ndarray):
    """
    Format the embeddings for clustering.
    """
    qid_lst: List[str] = []
    documents: List[str] = []
    embedding_lst: List[np.ndarray] = []

    qid_to_chapter = {}
    for chapter_id in qb.list_chapters():
        for qid in qb.get_qids_by_chapter(chapter_id):
            qid_lst.append(qid)
            qid_to_chapter[qid] = chapter_id
    qid_lst.sort()

    for qid in qid_lst:
        documents.append(
            format_question(qb.get_question(qid), qb.describe_chapter(
                qid_to_chapter[qid])))
        embedding_lst.append(id_to_embedding[qid])

    embedding_array = np.array(embedding_lst)
    return qid_lst, documents, embedding_array

In [20]:
qid_lst, documents, embeds = format_for_clustering(id_to_embedding, qb)
print(f"Number of questions: {len(qid_lst)} "
      f"Embedding shape: {embeds.shape}")

Number of questions: 2836 Embedding shape: (2836, 768)


In [31]:
documents[1500:1510]  # Display the first few documents to verify formatting

['章节:交通信号 题目:如图所示，驾驶机动车行驶至铁路道口时，以下做法正确的是什么？ 答案:停车等待',
 '章节:摩托车专用试题 题目:这个标志是何含义？ 答案:村庄或集镇',
 '章节:道路交通安全法律、法规和规章 题目:申请人以欺骗、贿赂等不正当手段取得校车驾驶资格的，处两千元以下罚款，申请人在三年内不得再次申请校车驾驶资格。 答案:对',
 '章节:交通信号 题目:以下哪个交通标志表示该道路仅供非机动车与行人通行，机动车不准进入? 答案:图D',
 '章节:交通信号 题目:图中这组交警手势是什么信号？ 答案:左转弯待转信号',
 '章节:交通信号 题目:这个标志是何含义？ 答案:步行',
 '章节:货车专用试题 题目:以下缩写中，表示车辆车道偏离预警系统缩写的是什么？ 答案:LDW',
 '章节:道路交通安全法律、法规和规章 题目:机动车驾驶人申请学法减分时，参加交通安全公益活动满一小时的，可扣减几分？ 答案:1分',
 '章节:交通信号 题目:如图所示，蓝色小型客车（C）行驶车道是正确的。 答案:对',
 '章节:摩托车专用试题 题目:会车中道路一侧有障碍，双方机动车应如何做？ 答案:有障碍的一方让对向先行']

### b) Set up dimension reduction model
We will be using UMAP for dimension reduction.

In [22]:
from umap import UMAP
def dim_reduction(dimensions: int, n_neighbors: int, embeddings: np.ndarray) -> np.ndarray:
    umap_model = UMAP(
        n_components=dimensions,
        metric='euclidean',
        n_neighbors=n_neighbors,
        min_dist=0.0
    )
    return umap_model.fit_transform(embeddings)

In [23]:
def get_reduced(dimension: int, n_neighbors: int, embeddings: np.ndarray, make_new: bool = False) -> np.ndarray:
    path = f"data_storage/embedding_dir/rdc_embeds_n{n_neighbors}d{dimension}.npz"
    if not os.path.exists(path) or make_new:
        rdc_embeds = dim_reduction(dimension, n_neighbors, embeddings)
        np.savez(path, embeddings=rdc_embeds)
    else:
        rdc_embeds = np.load(path)['embeddings']
    return rdc_embeds

## 5. Clustering


Generate a small representative sample of the question bank by clustering the questions and selecting representative questions from each cluster.

### a) Set up clustering model

In [24]:
from sklearn.cluster import HDBSCAN
def cluster_embeddings(embeddings: np.ndarray) -> np.ndarray:
    """
    Cluster the embeddings using HDBSCAN.

    Args:
        embeddings: Embeddings array with shape (n_samples, n_features)

    Returns:
        Cluster labels for each embedding
    """
    clusterer = HDBSCAN(min_cluster_size=2, metric='euclidean', min_samples=1, cluster_selection_method="leaf", cluster_selection_epsilon=0.15, allow_single_cluster=True)
    return clusterer.fit_predict(embeddings)

### b) Tune Hyperparameters
Find optimal hyperparameters
#### i) Set up experiments trying a range of hyperparameters

In [25]:
from pandas import DataFrame

In [26]:
def make_topics(dimension: int, n_neighbors: int, qid_lst: List[str], docs: List[str], embeddings: np.ndarray) -> DataFrame:
    """
    A pipeline that takes in the dimension and n_neighbors, then performs dimension reduction and clustering using those hyperparameters on docs.
    """
    topic_labels = cluster_embeddings(get_reduced(dimension, n_neighbors, embeddings))
    return DataFrame({
        "topic" : topic_labels,
        "qid": qid_lst,
        "question": docs
    })

In [27]:
from typing import Tuple
from ipywidgets import IntProgress, Label, HBox
from IPython.display import display

In [28]:
def dimred_experiment(qid_lst: List[str], questions: List[str], embeddings: np.ndarray, n_d_pairs: List[Tuple[int]]) -> DataFrame:
    """
    An experiment to find the optimal hyperparameters for dimension reduction (dimensions and n_neighbors).

    With a fixed clustering method, we will vary the dimensions and n_neighbors to see how it affects the outlier count.
    """

    cur, total = 0, len(n_d_pairs)
    progress_bar = IntProgress(min=0, max=total, description='Progress:')
    progress_label = Label(f'0/{total} complete')
    progress_box = HBox([progress_bar, progress_label])
    display(progress_box)

    result_dict = {
        "qid": qid_lst,
        "question": questions
    }
    for n_neighbors, dimension in n_d_pairs:
        progress_label.value = f"{cur}/{total} complete. Calculating n = {n_neighbors}, d = {dimension}..."

        column_name = f"n{n_neighbors}d{dimension}"
        result_dict[column_name] = cluster_embeddings(get_reduced(dimension, n_neighbors, embeddings))

        cur += 1
        progress_bar.value += 1
    progress_label.value = "Experiment complete!"
    return DataFrame(result_dict)

In [29]:
import pandas as pd
import warnings

In [30]:
DIMRED_EXPR1_PATH = "data_storage/experiments/dimension_reduction_experiment1.csv"
if not os.path.exists(DIMRED_EXPR1_PATH):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        n_d_pairs = [(n, d) for n in range(2, 767, 100) for d in range(2, 767, 100)]
        results = dimred_experiment(qid_lst=qid_lst,
                                    questions=documents,
                                    embeddings=embeds,
                                    n_d_pairs=n_d_pairs)
        results.to_csv(DIMRED_EXPR1_PATH, index=False)
        dimred_exp1_results = results
else:
    print(f"Results already exist at {DIMRED_EXPR1_PATH}. Loading results from file...")
    dimred_exp1_results = pd.read_csv(DIMRED_EXPR1_PATH)

In [31]:
combined_results = dimred_exp1_results
print(combined_results.shape)

(2836, 66)


#### ii) Analyze Results

Extract key metrics from the data into a dataframe.

In [32]:
def extract_n_d(col_name: str) -> (int, int):
    """
    Extract n_neighbors and dimensions from the column name.
    """
    parts = col_name.split("d")
    n_neighbors = int(parts[0][1:])  # Remove 'n' and convert to int
    dimensions = int(parts[1])  # Convert to int
    return n_neighbors, dimensions

In [33]:
def analyze_dimred_data(data: DataFrame) -> DataFrame:
    """
    Set up a data frame with n_neighbours and dimensions as the first 2 columns, followed by
    key metrics, such as:
        - Number of clusters
        - Number of outliers
        - ...
    """
    metrics = {
        "n_neighbors": [],
        "dimensions": [],
        "num_clusters": [],
        "num_outliers": []
    }
    for col in data.columns:
        if col.startswith("n") and "d" in col:
            n_neighbors, dimensions = extract_n_d(col)
            metrics["n_neighbors"].append(n_neighbors)
            metrics["dimensions"].append(dimensions)
            metrics["num_clusters"].append(data[col].nunique())
            metrics["num_outliers"].append((data[col] == -1).sum())
    return DataFrame(metrics)

In [34]:
import plotly.express as px

In [35]:
def interactive_plot3d(df, z_metric, size=(1600, 800)):
    """
    Creates an interactive 3D plot using Plotly.

    Parameters:
    -----------
    df : DataFrame
        DataFrame with 'n_neighbors', 'dimensions', and z_metric
    z_metric : str
        Column name to use for z-axis values
    size : tuple, optional
        Figure size in pixels
    """
    fig = px.scatter_3d(
        df,
        x='n_neighbors',
        y='dimensions',
        z=z_metric,
        color=z_metric,
        color_continuous_scale='Viridis',
        range_color=(df[z_metric].min(), df[z_metric].max()),
        title=f'3D Plot of {z_metric} by n_neighbors and dimensions'
    )
    fig.update_traces(marker=dict(size=6))
    fig.update_layout(
        width=size[0],
        height=size[1],
        scene=dict(
            xaxis_title='n_neighbors',
            yaxis_title='dimensions',
            zaxis_title=z_metric
        ),
        coloraxis_colorbar=dict(
            title=z_metric,
            len=0.75
        )
    )
    return fig

Examine number of clusters

In [36]:
metrics = analyze_dimred_data(combined_results)
fig = interactive_plot3d(metrics, z_metric='num_clusters')
fig.show()

Examine number of outliers

In [37]:
fig = interactive_plot3d(metrics, z_metric='num_outliers')
fig.show()

#### iii) Refine Experiment Based on Results
##### Experiment 2

In [38]:
def get_nd_experiment2() -> List[Tuple[int]]:
    """
    Get the n_neighbors and dimensions pairs for the second experiment.
    This is based on the results of the first experiment.
    """
    nd_set = set()
    for n in range(2, 50, 5):
        for d in range(2, 10, 1):
            nd_set.add((n, d))
        for d in range(10, 50, 5):
            nd_set.add((n, d))
    return list(nd_set)

In [39]:
def run_experiment(qid_lst: List[str], documents: List[str], embeds: np.ndarray, n_d_pairs: List[Tuple[int]], path: str) -> DataFrame:
    """Set up experiment."""
    # Suppress all warnings within this block
    if not os.path.exists(path):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            results = dimred_experiment(qid_lst=qid_lst, questions=documents, embeddings=embeds, n_d_pairs=n_d_pairs)
            results.to_csv(path, index=False)
    else:
        results = pd.read_csv(path)
    return results

In [40]:
DIMRED_EXPR2_PATH = "data_storage/experiments/dimension_reduction_experiment2.csv"

In [41]:
dimred_exp2_results = run_experiment(qid_lst=qid_lst, documents=documents, embeds=embeds, n_d_pairs=get_nd_experiment2(), path=DIMRED_EXPR2_PATH)

In [42]:
combined_results = pd.concat([
    combined_results,
    dimred_exp2_results.drop(columns=["qid", "question"])], axis=1).reindex()
combined_results.shape

(2836, 226)

In [43]:
metrics = analyze_dimred_data(dimred_exp2_results)

In [44]:
fig = interactive_plot3d(metrics, z_metric='num_clusters')
fig.show()

In [45]:
fig = interactive_plot3d(metrics, z_metric='num_outliers')
fig.show()

Inspect certain points manually to determine the quality of clusters

In [46]:
def inspect_topic(data: DataFrame, n_neighbors: int, dimensions: int, sort_df=False, ascending=False) -> DataFrame:
    """
    Inspect the results for a specific n_neighbors and dimensions combination.
    """
    col_name = f"n{n_neighbors}d{dimensions}"
    if col_name in data.columns:
        output = data[["qid", "question", col_name]].rename(columns={col_name: "topic"})
        if sort_df:
            output = output.sort_values(by="topic", ascending=ascending)
    else:
        raise ValueError(f"No results found for n_neighbors={n_neighbors} and dimensions={dimensions}")
    return output

In [47]:
def plot_2d_clusters(data: DataFrame, size: Tuple[int,int] = (1400,1200)) -> px.scatter:
    """
    Plot the 2D clusters and color code by topic

    ====== Representational Invariant ==========
    - embeddings are two dimensional
    """
    fig = px.scatter(
        data, x=0, y=1,
        color='topic',
        title='2D Clusters Colored by Topic',
        color_continuous_scale="Viridis",
        hover_name="question"
    )
    fig.update_traces(marker=dict(size=10))
    fig.update_layout(
        width=size[0],
        height=size[1],
        xaxis_title='X',
        yaxis_title='Y',
        plot_bgcolor='white'
    )
    return fig


def plot_3d_clusters(data: DataFrame, size: Tuple[int,int] = (1400,1000)) -> px.scatter_3d:
    """
    Plot the 3D clusters and color code by topic

    ====== Representational Invariant ==========
    - embeddings are three dimensional
    """
    fig = px.scatter_3d(
        data, x=0, y=1, z=2,
        color='topic',
        title='3D Clusters Colored by Topic',
        color_continuous_scale="Viridis",
        hover_name="question"
    )
    fig.update_traces(marker=dict(size=10))
    fig.update_layout(
        width=size[0],
        height=size[1],
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        )
    )
    return fig

In [48]:
from typing import Dict

def make_embeds_with_topics(qid_lst: List[str],
                            qb: QuestionBank,
                            topic_array: np.ndarray, reduced_embeddings: np.ndarray,
                            dimension: int) -> DataFrame:
    result_dict = {
        "qid": qid_lst,
        "question": [format_question(qb.get_question(qid)) for qid in qid_lst],
        "topic": topic_array
    }
    for i in range(dimension):
        result_dict[i] = reduced_embeddings[:, i]
    return DataFrame(result_dict)

def make_embeds_df(qid_lst: List[str],
                   qb: QuestionBank,
                   id_to_embedding: Dict[str, np.ndarray],
                   dim: int, n_nghbrs: int) -> DataFrame:
    embedding_lst = [id_to_embedding[qid] for qid in (sorted(qid_lst))]
    embeds_3d = get_reduced(dimension=dim,
                            n_neighbors=n_nghbrs,
                            embeddings=np.array(embedding_lst),
                            make_new=True)
    return make_embeds_with_topics(sorted(qid_lst),
                                   qb,
                                   cluster_embeddings(embeds_3d),
                                   embeds_3d,
                                   dim)

In [49]:
embeds_with_topics = make_embeds_df(combined_results["qid"].tolist(),
                                    qb,
                                    id_to_embedding,
                                    dim=2, n_nghbrs=5)

/Users/simonxu/opt/anaconda3/envs/drivetest_tag_gen/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [50]:
fig = plot_2d_clusters(embeds_with_topics)
# fig = plot_3d_clusters(embeds_with_topics)
fig.show()

In [51]:
embeds_with_topics.head(20)

,qid,question,topic,0,1
0,00428,题目:这个标志是何含义？ 答案:禁止直行和向右转弯,100,7.499687,-4.585305
1,00624,题目:这个标志是何含义？ 答案:十字交叉路口预告,124,6.762934,-1.283506
2,0079e,题目:这个标志是何含义？ 答案:会车时停车让对方车先行,100,7.482961,-4.264588
3,008da,题目:谨慎驾驶的三个原则是集中注意力、仔细观察和提前预防。 答案:对,126,0.403703,4.953165
4,0092c,题目:在这条公路上行驶的最高速度不能超过多少？ 答案:40公里/小时,142,17.299940,3.796474
5,0146a,题目:驾驶机动车在雾天行车开启雾灯和危险报警闪光灯。 答案:对,48,0.400715,6.128050
6,019e1,题目:驾驶机动车在路口遇到这种信号灯亮时，不能右转弯。 答案:错,136,17.949568,4.487716
7,01e32,题目:灯光开关旋转到这个位置时，全车灯光点亮。 答案:错,23,20.486418,8.022362
8,01ea3,题目:如图所示，这辆小轿车不能在这个位置停车。 答案:对,118,17.592196,1.434911
9,01fdd,题目:汽车列车在转弯过程中，要注意提前做好让车准备。 答案:对,3,5.004407,18.138884
